# Imports

In [2]:
import pandas as pd
import numpy as np
import os
from datasets import load_dataset
from datasets import Dataset
from tqdm.auto import tqdm
import re
import glob
tqdm.pandas()

This notebook will create the final dataset for the project. The book datasets will be merged from source [1], [2] to [3] to give use the final book dataset. The users will be used as is from [3].

# Books

The folder `2_final` has parquet files having the books from [2] with the matching ID from [1] and [3]. We will use them to create a final book dataset.

The idea is to use the author and genre information from [1] and the rest of the information from [2] for books. These will then be merged with [3] to give us a final book dataset along with the ratings of the books for different users.

## Loading Data and Preprocessing

### Books from [1]

Some preprocessing needs to be done before we can merge the data. This section will deal with it.

In [2]:
books1 = pd.read_parquet("1_2/all_books_with_id.parquet")
books1 = Dataset.from_pandas(books1)

In [3]:
books1[1]

{'authors': '[2, 3]',
 'categories': '[235, 3386]',
 'format': 1.0,
 'isbn10': '184454737X',
 'lang': 'en',
 'publication-date': '2009-03-13 00:00:00',
 'title': 'Underbelly : The Gangland War',
 'authors_': 'John Silvester: Andrew Rule',
 'ID': 1}

Let's remove the following columns, as they will not be used later:
- authors_
- isbn10 ([3] will be used)
- title (will be used from [2])
- publication-date (will be used from [2])

In [4]:
columns_to_remove = ['isbn10', 'publication-date', 'title', 'authors_']
books1 = books1.remove_columns(columns_to_remove)
books1

Dataset({
    features: ['authors', 'categories', 'format', 'lang', 'ID'],
    num_rows: 1109383
})

In [5]:
books1[1213]

{'authors': '[631]',
 'categories': '[2978, 2452, 3385]',
 'format': 9.0,
 'lang': 'en',
 'ID': 1213}

The `ID` column will be used to match the books. It is a simple integer, so it can be used as the index.

### Books from [2]

Books from [2] are divided into many files. Let's use a sample file to work with.

In [6]:
books2 = pd.read_parquet(f"2_final/book400k-500k.parquet")
books2 = Dataset.from_pandas(books2)
books2

Dataset({
    features: ['id', 'title', 'authors', 'pages', 'pages.1', 'language', 'star_1', 'star_2', 'star_3', 'star_4', 'star_5', 'rating', 'total_ratings', 'total_reviews', 'isbn', 'publication_date', 'title_', 'series', 'book_num', 'book_id', 'score', 'text', 'book_id_3', 'score_3'],
    num_rows: 8169
})

In [7]:
books2[5]

{'id': 400054,
 'title': 'One Day in the Desert',
 'authors': 'Jean Craighead George',
 'pages': 64,
 'pages.1': 64,
 'language': None,
 'star_1': 2,
 'star_2': 4,
 'star_3': 24,
 'star_4': 26,
 'star_5': 17,
 'rating': 3.71,
 'total_ratings': 73,
 'total_reviews': 14,
 'isbn': '0064420388',
 'publication_date': '1996-04-12',
 'title_': 'One Day in the Desert',
 'series': 'Standalone',
 'book_num': 1,
 'book_id': 1003093,
 'score': 7.20892333984375,
 'text': 'One Day in the Desert by Jean Craighead George',
 'book_id_3': -1,
 'score_3': 16.608840942382812}

We will be dropping the following columns:
- pages.1 (Duplicate of pages)
- authors (To be used from [1])
- language (most of them are Null, we will use [1] to get the language)
- title (the columns title_ and series will be used instead)
- text (redundant column)
- isbn (will be used from [3])

In [8]:
columns_to_remove = ["title", "pages.1", "language", "text", "authors", "isbn"]
books2 = books2.remove_columns(columns_to_remove)
books2

Dataset({
    features: ['id', 'pages', 'star_1', 'star_2', 'star_3', 'star_4', 'star_5', 'rating', 'total_ratings', 'total_reviews', 'publication_date', 'title_', 'series', 'book_num', 'book_id', 'score', 'book_id_3', 'score_3'],
    num_rows: 8169
})

We will also need to rename some column name:

In [9]:
rename_map = {
    "title_": "title",
    "book_id": "book_id_1",
    "score": "score_1",
}

for key, value in rename_map.items():
    books2 = books2.rename_column(key, value)

In [10]:
books2

Dataset({
    features: ['id', 'pages', 'star_1', 'star_2', 'star_3', 'star_4', 'star_5', 'rating', 'total_ratings', 'total_reviews', 'publication_date', 'title', 'series', 'book_num', 'book_id_1', 'score_1', 'book_id_3', 'score_3'],
    num_rows: 8169
})

We will create a function for all these steps as we will need to repeat this for many datasets.

In [11]:
def load_and_preprocess_2(file_dir):
    books2 = pd.read_parquet(file_dir)
    books2 = Dataset.from_pandas(books2)

    columns_to_remove = ["title", "pages.1", "language", "text", "authors"]
    try:
        books2 = books2.remove_columns(columns_to_remove)
    except ValueError:
        columns_to_remove.pop(1)
        books2 = books2.remove_columns(columns_to_remove)
    rename_map = {
        "title_": "title",
        "book_id": "book_id_1",
        "score": "score_1",
    }

    for key, value in rename_map.items():
        books2 = books2.rename_column(key, value)
    return books2

In [12]:
books2 = load_and_preprocess_2("2_final/book400k-500k.parquet")
books2[10]

{'id': 400074,
 'pages': 6,
 'star_1': 301,
 'star_2': 1213,
 'star_3': 5878,
 'star_4': 10626,
 'star_5': 7925,
 'rating': 3.95,
 'total_ratings': 25943,
 'total_reviews': 16,
 'isbn': '1572704438',
 'publication_date': '2008-01-22',
 'title': 'The Thin Man',
 'series': 'Standalone',
 'book_num': 1,
 'book_id_1': 364146,
 'score_1': 0.0,
 'book_id_3': 20735,
 'score_3': 1.9559478759765625}

### Books from [3]

In [13]:
books3 = load_dataset("parquet", data_files="3_clean/books_with_id_and_embedding.parquet")

In [14]:
books3 = books3["train"]
books3

Dataset({
    features: ['isbn', 'title', 'authors', 'publication_year', 'text', 'text_embedding', 'ID'],
    num_rows: 271360
})

We will be removing the following columns:
- text (redundant)
- text_embedding (redundant)
- authors (to be used from [1])
- publication_year

We will keep the `title` column for the time being, so that we can verify the final dataset. Later, it will be dropped in favor of `title` from [2].

In [15]:
columns_to_remove = ["text", "text_embedding", "authors", "publication_year"]
books3 = books3.remove_columns(columns_to_remove)
books3

Dataset({
    features: ['isbn', 'title', 'ID'],
    num_rows: 271360
})

In [16]:
books3 =books3.rename_column("title", "title3")

In [17]:
books3[1]

{'isbn': '0002005018', 'title3': 'Clara Callan', 'ID': 1}

The `isbn` column will be used to map with the users.

## Merging the Dataframes

The books from [2] has the IDs for the matching book in [1] and [3]. If no match is found, we have imputed -1. Also, this ID is nothing but the index of the book in the dataframe [1] and [3], if not shuffled. Using these, the next section will deal with merging the all three dataframes into one.

We will start by filtering out only those rows in [2] where a match has been found for both [1] and [3]:

In [18]:
def filter_function(row):
    id_1 = row["book_id_1"]
    id_3 = row["book_id_3"]
    if id_1 == -1 or id_3 == -1:
        return False
    return True

In [19]:
books2_filtered = books2.filter(filter_function)
print(f"Number of books before filtering: {len(books2)}")
print(f"Number of books after filtering: {len(books2_filtered)}")

Filter:   0%|          | 0/8169 [00:00<?, ? examples/s]

Number of books before filtering: 8169
Number of books after filtering: 2497


It seems the just about 30% books are matching in both the datasets. We have to make work with this data only.

In [20]:
sample_book_id = 1212
book2_detail = books2_filtered[sample_book_id]
book2_detail

{'id': 439490,
 'pages': 670,
 'star_1': 41593,
 'star_2': 74409,
 'star_3': 248277,
 'star_4': 481402,
 'star_5': 712095,
 'rating': 4.12,
 'total_ratings': 1557776,
 'total_reviews': 4,
 'isbn': '8497593456',
 'publication_date': '2003-02-28',
 'title': 'Jane Eyre',
 'series': 'Standalone',
 'book_num': 1,
 'book_id_1': 255627,
 'score_1': 0.0,
 'book_id_3': 3323,
 'score_3': 0.0}

In [21]:
def match_book(row):
    book1_id = row["book_id_1"]
    book3_id = row["book_id_3"]
    info_from_book1 = books1[book1_id]
    info_from_book3 = books3[book3_id]
    info_from_book1.pop("ID")
    info_from_book3.pop("ID")
    final_info = {**row, **info_from_book1, **info_from_book3}
    return final_info

In [22]:
books2_matched = books2_filtered.map(match_book)

Map:   0%|          | 0/2497 [00:00<?, ? examples/s]

In [23]:
pd.DataFrame(books2_matched[:5]).T

,0,1,2,3,4
id,400019,400074,400082,400083,400096
pages,304,6,192,111,92
star_1,5,301,43,30,117
star_2,16,1213,100,71,313
star_3,106,5878,309,284,1233
star_4,128,10626,406,457,1692
star_5,86,7925,381,429,1393
rating,3.8,3.95,3.79,3.93,3.83
total_ratings,341,25943,1239,1271,4748
total_reviews,54,16,12,9,9


This is working as intended. Let's make a final function to be used for each dataframe from [2].

In [24]:
to_save_dir = "merged_dataset"

In [25]:
def match_one_book_df(file_dir):
    books2 = load_and_preprocess_2(file_dir)
    books2_filtered = books2.filter(filter_function)
    print(f"Number of books before filtering: {len(books2)}")
    print(f"Number of books after filtering: {len(books2_filtered)}")
    books2_matched = books2_filtered.map(match_book)
    file_name = file_dir.split(os.path.sep)[-1]
    file_name = os.path.join(to_save_dir, file_name)
    books2_matched.to_parquet(file_name)
    print(f"Saved to {file_name}")

In [26]:
all_books_dir = glob.glob("2_final/book*.parquet")
all_books_dir.sort()
len(all_books_dir)

22

In [27]:
start_id = 0
for current_id in range(start_id, len(all_books_dir)):
    print(f"CURRENTLY WORKING ON: {current_id}")
    match_one_book_df(all_books_dir[current_id])
    print("----"*10)
    print("----"*10)

CURRENTLY WORKING ON: 0


Filter:   0%|          | 0/16146 [00:00<?, ? examples/s]

Number of books before filtering: 16146
Number of books after filtering: 7718


Map:   0%|          | 0/7718 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

...

Filter:   0%|          | 0/7325 [00:00<?, ? examples/s]

Number of books before filtering: 7325
Number of books after filtering: 2676


Map:   0%|          | 0/2676 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Saved to merged_dataset\book900k-1000k.parquet
----------------------------------------
----------------------------------------


## Creating a Single Dataframe

We will be creating a single big dataframe for all the books. This may later be cleaned and post-processed to create a final dataset.

In [3]:
all_books_dir = glob.glob("merged_dataset/book*.parquet")
all_books_dir.sort()
len(all_books_dir)

22

In [23]:
rename_map = {
    "pages": "num_pages",
    "star_1": "star_rating_1",
    "star_2": "star_rating_2",
    "star_3": "star_rating_3",
    "star_4": "star_rating_4",
    "star_5": "star_rating_5",
    "rating": "average_rating",
    "book_num": "book_number_in_series",
}
columns_to_drop = [
        'id', 'book_id_1', 'score_1', 'book_id_3', 'score_3', "title3"
    ]

In [24]:
def read_and_preprocess(file_dir):
    df = pd.read_parquet(file_dir)
    df = df.drop(columns_to_drop, axis=1)
    df["publication_date"] = df["publication_date"].apply(lambda x: x.split(" ")[0])
    df = df.rename(columns=rename_map)
    return df

In [30]:
dfs = []
for file_path in tqdm(all_books_dir, desc='Reading and preprocessing books'):
    df = read_and_preprocess(file_path)
    print(df.shape)
    dfs.append(df)
final_df = pd.concat(dfs, ignore_index=True)

Reading and preprocessing books:   0%|          | 0/22 [00:00<?, ?it/s]

(7718, 18)
(2165, 18)
(4198, 18)
(2173, 18)
(1900, 18)
(1568, 18)
(1521, 18)
(1049, 18)
(1019, 18)
(1087, 18)
(1248, 18)
(1284, 18)
(7883, 18)
(3351, 18)
(4126, 18)
(2412, 18)
(2497, 18)
(2979, 18)
(2244, 18)
(2333, 18)
(3381, 18)
(2676, 18)


In [31]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60812 entries, 0 to 60811
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   num_pages              60812 non-null  int64  
 1   star_rating_1          60812 non-null  int64  
 2   star_rating_2          60812 non-null  int64  
 3   star_rating_3          60812 non-null  int64  
 4   star_rating_4          60812 non-null  int64  
 5   star_rating_5          60812 non-null  int64  
 6   average_rating         60812 non-null  float64
 7   total_ratings          60812 non-null  int64  
 8   total_reviews          60812 non-null  int64  
 9   isbn                   60812 non-null  object 
 10  publication_date       60812 non-null  object 
 11  title                  60812 non-null  object 
 12  series                 60812 non-null  object 
 13  book_number_in_series  60812 non-null  int64  
 14  authors                60812 non-null  object 
 15  ca

It seems we have about 60k books that are matching out of over 200k books that we started with. I think we have make work with this.

In [32]:
final_df.to_parquet('merged_dataset/all_books.parquet')